In [2]:
import torch
device= 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

block_size=8
batch_size=4

cuda


In [3]:
with open('wizard.txt', 'r', encoding= 'utf-8') as f:
    text= f.read()
    
print(len(text))

233437


In [4]:
print(text[:200])

  DOROTHY AND THE WIZARD IN OZ

  BY

  L. FRANK BAUM

  AUTHOR OF THE WIZARD OF OZ, THE LAND OF OZ, OZMA OF OZ, ETC.

  ILLUSTRATED BY JOHN R. NEILL

  BOOKS OF WONDER WILLIAM MORROW & CO., INC. NEW


In [5]:
chars= sorted(set(text))

In [6]:
print(chars)

['\n', ' ', '!', '"', '#', '&', "'", '(', ')', '*', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']


In [7]:
print(len(chars))

83


In [8]:
string_to_int= { ch: i for i,ch in enumerate(chars) }
int_to_string= { i: ch for i,ch in enumerate(chars) }

encode= lambda s: [string_to_int[c] for c in s]
decode= lambda l: ''.join([int_to_string[i] for i in l])


In [9]:
encoded_hello= encode("hello")
print(encoded_hello)
decoded_hello= decode(encoded_hello)
print(decoded_hello)

[63, 60, 67, 67, 70]
hello


In [10]:
data= torch.tensor(encode(text), dtype= torch.long)

In [11]:
data.shape

torch.Size([233437])

In [12]:
n= int(0.8*len(data))
train_data= data[:n]
val_data= data[n:]

In [13]:
block_size= 8

x= train_data[:block_size]
y= train_data[1: block_size+1]

for t in range(block_size):
    context= x[:t+1]
    target= y[t]
    print(f"when input is {context} target is {target}")

when input is tensor([82]) target is 1
when input is tensor([82,  1]) target is 1
when input is tensor([82,  1,  1]) target is 30
when input is tensor([82,  1,  1, 30]) target is 41
when input is tensor([82,  1,  1, 30, 41]) target is 44
when input is tensor([82,  1,  1, 30, 41, 44]) target is 41
when input is tensor([82,  1,  1, 30, 41, 44, 41]) target is 46
when input is tensor([82,  1,  1, 30, 41, 44, 41, 46]) target is 34


#### CPU VS GPU runtime

In [14]:
import time

In [15]:
torch_rand1= torch.rand(10000, 10000).to(device)
torch_rand2= torch.rand(10000, 10000).to(device)

np_rand1= torch.rand(10000, 10000)
np_rand2= torch.rand(10000, 10000)

start_time= time.time()
rand= torch_rand1 @ torch_rand2
end_time= time.time()

elapsed_time= end_time-start_time

print(f"Time Elapsed: {elapsed_time}")

start_time= time.time()
rand= np_rand1 @ np_rand2
end_time= time.time()

elapsed_time= end_time-start_time

print(f"Time Elapsed: {elapsed_time}")



Time Elapsed: 0.03391218185424805
Time Elapsed: 8.403530359268188


In [20]:
probabilities= torch.tensor([0.1, 0.9])
# 0 => 0 index with 10% prob and 1 index with 90% prob
samples= torch.multinomial(probabilities, num_samples=20, replacement=True)
print(samples)

tensor([0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0])


In [21]:
triangle_lower= torch.tril(torch.ones(5,5))
print(triangle_lower)

tensor([[1., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0.],
        [1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1.]])


In [22]:
triangle_upper= torch.triu(torch.ones(5,5))
print(triangle_upper)

tensor([[1., 1., 1., 1., 1.],
        [0., 1., 1., 1., 1.],
        [0., 0., 1., 1., 1.],
        [0., 0., 0., 1., 1.],
        [0., 0., 0., 0., 1.]])


In [24]:
masked_triangle_lower= torch.zeros(5,5).masked_fill(torch.tril(torch.ones(5,5))==0, float('-inf'))
print(masked_triangle_lower)

tensor([[0., -inf, -inf, -inf, -inf],
        [0., 0., -inf, -inf, -inf],
        [0., 0., 0., -inf, -inf],
        [0., 0., 0., 0., -inf],
        [0., 0., 0., 0., 0.]])


In [25]:
import torch.nn as nn 
sample= torch.tensor([10.,10.,10.])
linear= nn.Linear(3,3,bias= False)
print(linear(sample))

tensor([ 8.5744,  1.7964, 12.6962], grad_fn=<SqueezeBackward4>)


In [26]:
import torch.nn.functional as F 
tensor1= torch.tensor([1.0, 2.0, 3.0])
softmax_ouput= F.softmax(tensor1, dim=0)

print(softmax_ouput)

tensor([0.0900, 0.2447, 0.6652])


In [27]:
vocab_size= 1000
embedding_dim= 100
embedding= nn.Embedding(vocab_size, embedding_dim)

# Create some input indices 
input_indices= torch.LongTensor([1,5,3,2])
embedded_ouput= embedding(input_indices)

print(embedded_ouput.shape)

torch.Size([4, 100])


In [31]:
n= int(0.8*len(data))
train_data= data[:n]
val_data= data[n:]

def get_batch(split):
    data= train_data if split=="train" else val_data 
    ix= torch.randint(len(data)-block_size, (batch_size,))
    print(ix)
    x= torch.stack([data[i: i+block_size] for i in ix])
    y= torch.stack([data[i+1: i+block_size+1] for i in ix])
    x,y= x.to(device), y.to(device)
    return x,y 

x,y= get_batch("train")
print("inputs: ")
print(x)
print("targets")
print(y)

tensor([82317, 39334, 90593, 18290])
inputs: 
tensor([[76, 69, 75, 73, 80, 12,  0,  0],
        [ 0,  0,  3, 34, 60, 73, 60,  1],
        [60, 56, 73, 10,  3,  1, 56, 69],
        [78, 67, 80,  1, 70, 77, 60, 73]], device='cuda:0')
targets
tensor([[69, 75, 73, 80, 12,  0,  0,  3],
        [ 0,  3, 34, 60, 73, 60,  1, 64],
        [56, 73, 10,  3,  1, 56, 69, 74],
        [67, 80,  1, 70, 77, 60, 73,  1]], device='cuda:0')


In [32]:
from torch.nn import functional as F

In [36]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table= nn.Embedding(vocab_size, vocab_size)
        
    def forward(self, index, targets=None):
        logits= self.token_embedding_table(index)

        if targets is None:
            loss= None 
        else:
            B,T,C= logits.shape
            logits= logits.view(B*T,C)
            targets= targets.view(B*T)
            loss= F.cross_entropy(logits,targets)
        
        return logits, loss 
        
    def generate(self, index, max_new_tokens):
        for _ in range(max_new_tokens):
            # get preds
            logits, loss= self.forward(index)
            # focus only on the last time stamp 
            logits= logits[:, -1, :] # becomes (B,C)
            # apply softmax to get probs 
            probs= F.softmax(logits, dim=-1) # (B,C)
            # sample from distribution 
            index_next= torch.multinomial(probs, num_samples=1) # (B,1)
            # append sampled index to the running sequence 
            index= torch.cat((index, index_next), dim=1) # (B,T+1)
        return index 
    
model= BigramLanguageModel(vocab_size)
m= model.to(device)

context= torch.zeros((1,1), dtype= torch.long, device= device)
generated_chars= decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)    

KeyError: 643

In [37]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 8
batch_size = 4
max_iters = 1000
# eval_interval = 2500
learning_rate = 3e-4
eval_iters = 250

cuda


In [39]:
with open('wizard.txt', 'r', encoding='utf-8') as f:
    text = f.read()
chars = sorted(set(text))
print(chars)
vocab_size = len(chars)

['\n', ' ', '!', '"', '#', '&', "'", '(', ')', '*', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']


In [40]:
string_to_int = { ch:i for i,ch in enumerate(chars) }
int_to_string = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)
# print(data[:100])

In [41]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

x, y = get_batch('train')
print('inputs:')
# print(x.shape)
print(x)
print('targets:')
print(y)

inputs:
tensor([[ 0,  0,  3, 35,  1, 59, 70, 69],
        [60, 58, 75, 74, 12,  0,  0, 40],
        [60, 75, 74,  1, 56, 69, 59,  1],
        [75,  1, 73, 60, 68, 56, 64, 69]], device='cuda:0')
targets:
tensor([[ 0,  3, 35,  1, 59, 70, 69,  6],
        [58, 75, 74, 12,  0,  0, 40, 70],
        [75, 74,  1, 56, 69, 59,  1, 57],
        [ 1, 73, 60, 68, 56, 64, 69, 60]], device='cuda:0')


In [42]:
# class BigramLanguageModel(nn.Module):
#     def __init__(self, vocab_size):
#         super().__init__()
#         self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        
#     def forward(self, index, targets=None):
#         logits = self.token_embedding_table(index)
        
        
#         if targets is None:
#             loss = None
#         else:
#             B, T, C = logits.shape
#             logits = logits.view(B*T, C)
#             targets = targets.view(B*T)
#             loss = F.cross_entropy(logits, targets)
        
#         return logits, loss
    
#     def generate(self, index, max_new_tokens):
#         # index is (B, T) array of indices in the current context
#         for _ in range(max_new_tokens):
#             # get the predictions
#             logits, loss = self.forward(index)
#             # focus only on the last time step
#             logits = logits[:, -1, :] # becomes (B, C)
#             # apply softmax to get probabilities
#             probs = F.softmax(logits, dim=-1) # (B, C)
#             # sample from the distribution
#             index_next = torch.multinomial(probs, num_samples=1) # (B, 1)
#             # append sampled index to the running sequence
#             index = torch.cat((index, index_next), dim=1) # (B, T+1)
#         return index

# model = BigramLanguageModel(vocab_size)
# m = model.to(device)

# context = torch.zeros((1,1), dtype=torch.long, device=device)
# generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
# print(generated_chars)


kZZQlvInLIJ*m3sWnYJzqb9E?quSveNwP8cP)foodyW2?c5.E.﻿A[*mCyR:;BtQMPAn4 B.ErH-9PXz(v]].z
iyco0tmZ[Yp/u0Az4'"C?E0N&V0k*(h﻿O&iM?;/Nj"Wnk3T( f"WcR]Vo:t;KcDqK!,RN0(ijvcDI/i ]nx4?;p;[Ux?aU;p7e﻿L*]nJ_h0&K8H:V﻿!rLiBh﻿Yp4arF&iUx j8XkcvAu﻿#cgfQL5uy!Dq[Uie'db9t3[pi'AgGR]s.G&*iI/oth!kPdClYd&A(FSMn2oXwNN4?GKO0(2mV?4
geIQFIpd.,6*;4ekzhUr*[M_qO1!j8N)Q3f)V95p5[?8iD)R-r0u o[ql:JN/mc4OT":tKnva U
k#vABT;LU6&wsxBh0;F:UuG3Li9mUaf-t1﻿YMYgc)oN/wNfiSd4'AMV?*rKxHF*hp7A8)i&hrwmwOlYdg4'A&tkSS4?Z6eT;)"*2JN&CJk3y!A!XanHLdAR"C
